# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
#CityData.csv
vacation_data = pd.read_csv('../WeatherPy/CityData.csv')
vacation_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,11.08,85,90,4.61,US,1612763527
1,yeppoon,-23.1333,150.7333,91.40,55,75,19.57,AU,1612763738
2,usinsk,65.9939,57.5281,-16.80,87,100,15.70,RU,1612763738
3,innisfail,-17.5333,146.0333,86.61,72,98,6.35,AU,1612763738
4,zhelyabovka,45.3988,34.7578,34.75,94,100,13.15,UA,1612763738


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key=g_key)
locations = vacation_data[['Lat', 'Lng']]
humidity = vacation_data['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                               max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
ideal_weather = vacation_data[(vacation_data['Max Temp'] < 85) & 
                              (vacation_data['Max Temp'] > 65) &
                              (vacation_data['Wind Speed'] < 10) &
                              (vacation_data['Cloudiness'] == 0) ]
vacation_data.head(5)                              

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,11.08,85,90,4.61,US,1612763527
1,yeppoon,-23.1333,150.7333,91.40,55,75,19.57,AU,1612763738
2,usinsk,65.9939,57.5281,-16.80,87,100,15.70,RU,1612763738
3,innisfail,-17.5333,146.0333,86.61,72,98,6.35,AU,1612763738
4,zhelyabovka,45.3988,34.7578,34.75,94,100,13.15,UA,1612763738


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = ideal_weather['Hotel Name' = ""]
hotel_df

SyntaxError: invalid syntax (<ipython-input-27-5dab950786e7>, line 1)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
